# Review: Glow: Generative Flow with Invertible 1×1 Convolutions

## Paper Reviews

### Prior Reseach

**Flow-based generative models**는 non-parametric density estimation의 일종으로 제안된 **normalizing flow**를 기반으로 하는 generative models임
+ density estimation의 일종으로서 variational inference는 기본적으로 lower bound에 대한 approximation이기 때문에 true distribution에 대한 tight estimation이 불가능함
+ 그러나 만약에 target density가 **적절한 $K$개의 transform의 chain이 존재하여** 간단한 distribution에서의 mapping으로 표현할 수 있다면 explicit하게 표현할 방법이 존재함  
+ chain을 구성하는 각각의 transform이 **invertible**하다면 흔히 [**LOTUS**](https://en.wikipedia.org/wiki/Law_of_the_unconscious_statistician)라 불리는 Theorem을 사용하는 것을 통해 target density를 몰라도 정확한 expectation을 계산해낼 수 있음  

$$ \mathbb{E}_{\mathbf{z_K} \sim q_K}[h(\mathbf{z_K})] = \mathbb{E}_{\mathbf{z_0} \sim q_0}[h(f_K \circ f_{K-1} \circ \cdots f_1(\mathbf{z}_0))] $$

→ $K$ chain이 infinite sequence고 **Langevin SDE**를 따르면 score function $ -\nabla_{\mathbf{z}} \mathcal{L}(\mathbf{z}) $로 stationary solution의 존재를 보장할 수 있음 ( = **diffusion model**)

이러한 approach는 기존의 dominant한 generative models(GAN, VAE, Autoregressive)에 비교해 다음과 같은 장점을 지니고 있음
+ explicit하게 log-likelihood을 통한 latent space에서부터의 target density의 inference를 수행하므로 VAE보다 정확한 inference가 가능해짐
+ Autoregressive와 같이 순차적으로 생성하는 경우보다 sampling speed가 유리함
+ semantic을 파악하기 힘든 marginal distribution을 생성하는 Autoregressive나 latent의 해석이 어려운 GAN보다 direct하게 latent space 상에서의 이해가 가능함
+ 특성상 Jacobian의 계산이 편해야 하는 구조이기 때문에 gradient의 계산이 쉬워 memory capacity 측면에서 다른 model보다 유리함

직접적인 model architecture 측면에서의 이를 위한 대표적인 선행연구는 **RealNVP**가 있음  
RealNVP는 **affine coupling layer**를 도입하는 것을 통해서 normalizing flow로서 만족하기 위한 invertible mapping 및 간단한 Jacobian을 얻어낼 수 있었음  

$$ y_{1:d} = x_{1:d} $$
$$ y_{d+1:D} = x_{d+1 : D} \odot \exp{(s(x_{1:d}))} + t(x_{1:d}) $$

그러나 단순히 affine coupling layer를 사용하면 identity mapping을 수행하는 channel에 대해서 transform이 이루어지지 않음  
따라서 RealNVP는 직접적으로 shuffling을 수행하는 과정을 추가하여 각 channel에 대한 pattern 변화를 반드시 도입했어야 했음  

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/181778517-f45945fa-bc87-4519-a0a4-04de9d4df6b0.png" alt="1" width="300px" />
</p>

이러한 측면을 개선하여 더 좋은 mapping을 학습할 수 있도록 **learnable shuffle**을 도입한 것이 **Glow**  
RealNVP에 비해서 image generation에서 훨씬 개선된 negative log-likelihood를 확인할 수 있었음  

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/181778523-6b47e380-787a-4fa1-8a3a-9c19942caa74.png" alt="2" width="500px" />
</p>

### Invertible 1×1 Convolutions

Glow에서 제시하는 learnable shuffling은 1×1 Convolutions이 **일종의 일반화된 permutation을 수행하는 것**이라는 점을 주장함  
+ 만약 1×1 Convolutions layer의 input/output channel의 dimension이 같다면 각 input channel의 일부를 linear combination하여 output의 크기 만큼의 channel로 반환함  
+ input channel의 order를 바꾸는 permutation을 넘어서 **각 input channel을 적절히 mixing하여 shuffling하여 같은 dimension의 output으로 반환하는 permutation**인 것  
+ 이러한 1×1 Convolutions shuffling으로 기존의 shuffling을 대체한다고 해도 충분히 쉽게 Jacobian을 구할 수 있으며 learnable하기까지 함  
+ 만약 $ c $ channel을 갖는 $ h \times w $ feature map $ \mathbf{h} $ 을 생각하고 1×1 kernel $ \mathbf{W} $를 고려하면 다음과 같이 쉽게 Jacobian을 구할 수 있음  

$$ \log{|\det{(\frac{d \, \text{conv2D}(\mathbf{h}; \mathbf{W})}{d \, \mathbf{h}})}|} = h \cdot w \cdot \log{|\det(\mathbf{W})|}$$ 

추가로 저자들은 $ \det(\mathbf{W}) $ 의 연산량인 $ \mathcal{O}(c^3) $ 를 줄이기 위해서 $ \mathbf{W} $ 의 **LU decompotion**을 학습하는 방법으로 $ \mathbf{W} $ 를 학습  
LU decomposition을 사용하는 것으로 permutation matrix $ P $를 통해 $ \mathbf{W} = \mathbf{PL}(\mathbf{U} + \text{diag}(\mathbf{s})) $ 로 표현할 수 있어짐  
이는 결과적으로 $ \log{|\det(\mathbf{W})|} = \text{sum}(\log{|\mathbf{s}|}) $ 라 단순한 vector sum으로 구할 수 있어지므로 연산량을 $ \mathcal{O}(c) $ 로 줄일 수 있음

최종적으로 RealNVP에 존재하던 Batch Normalization을 각 channel마다 수행하는 **Activation Normalization**으로 대체한 하나의 block을 구성  
이러한 block을 여러번의 stack을 통하여 최종적인 model architecture로 구성함

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/181778533-53389880-928d-496e-b94f-c48a96ee24df.png" alt="3" width="400px" />
</p>

## Implementation Reviews

+ paper의 중요 contribution인 [**1×1 Convolutions**](https://github.com/ikostrikov/pytorch-flows/blob/master/flows.py) implementation



In [ ]:
class LUInvertibleMM(nn.Module):
    """ An implementation of a invertible matrix multiplication
    layer from Glow: Generative Flow with Invertible 1x1 Convolutions
    (https://arxiv.org/abs/1807.03039).
    """

    def __init__(self, num_inputs):
        super(LUInvertibleMM, self).__init__()
        self.W = torch.Tensor(num_inputs, num_inputs)
        nn.init.orthogonal_(self.W)
        self.L_mask = torch.tril(torch.ones(self.W.size()), -1)
        self.U_mask = self.L_mask.t().clone()

        P, L, U = sp.linalg.lu(self.W.numpy())
        self.P = torch.from_numpy(P)
        self.L = nn.Parameter(torch.from_numpy(L))
        self.U = nn.Parameter(torch.from_numpy(U))

        S = np.diag(U)
        sign_S = np.sign(S)
        log_S = np.log(abs(S))
        self.sign_S = torch.from_numpy(sign_S)
        self.log_S = nn.Parameter(torch.from_numpy(log_S))

        self.I = torch.eye(self.L.size(0))

    def forward(self, inputs, cond_inputs=None, mode='direct'):
        if str(self.L_mask.device) != str(self.L.device):
            self.L_mask = self.L_mask.to(self.L.device)
            self.U_mask = self.U_mask.to(self.L.device)
            self.I = self.I.to(self.L.device)
            self.P = self.P.to(self.L.device)
            self.sign_S = self.sign_S.to(self.L.device)

        L = self.L * self.L_mask + self.I
        U = self.U * self.U_mask + torch.diag(
            self.sign_S * torch.exp(self.log_S))
        W = self.P @ L @ U

        if mode == 'direct':
            return inputs @ W, self.log_S.sum().unsqueeze(0).unsqueeze(
                0).repeat(inputs.size(0), 1)
        else:
            return inputs @ torch.inverse(
                W), -self.log_S.sum().unsqueeze(0).unsqueeze(0).repeat(
                    inputs.size(0), 1)

## Reference

https://arxiv.org/pdf/1807.03039.pdf  
https://arxiv.org/pdf/1505.05770.pdf  
https://arxiv.org/pdf/2011.13456.pdf  
https://github.com/ikostrikov/pytorch-flows/blob/master/flows.py  
